In [1]:
from neo4j import GraphDatabase

class Neo4jService(object):

    def __init__(self, uri, user, password):
        self._driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self._driver.close()      
    
    def crear_juego(self, tx, nombre):
        tx.run("CREATE (:juego {nombre: $nombre})", nombre=nombre)
        
    def crear_jugador(self, tx, nombre):
        tx.run("CREATE (:jugador {nombre: $nombre,puntuacion: $puntuacion,nivel:$nivel })", nombre=nombre,puntuacion=puntuacion,nivel=nivel)
        
    #realciones
    
    def crear_relacion_JuegoUS(self, tx, nombre_j, nombre_u):
        tx.run("MATCH (a:juego {nombre: $nombre_j}) "
               "MATCH (b:jugador {nombre: $nombre_u}) "
               "MERGE (b)-[:JUEGA]->(a)",
               nombre_j=nombre_j, nombre_u=nombre_u)
        
print("Ejecucion correcta")

Ejecucion correcta


In [5]:
neo4j = Neo4jService('bolt://localhost:7687', 'neo4j', 'cuenca')
with neo4j._driver.session() as session:
    session.write_transaction(neo4j.crear_juego , 'Bloques')

ServiceUnavailable: Failed to establish connection to IPv4Address(('127.0.0.1', 7687)) (reason [WinError 10061] No se puede establecer una conexión ya que el equipo de destino denegó expresamente dicha conexión)

In [4]:
  
import pygame
import random
import requests
from bs4 import BeautifulSoup
  

    
# Definimos algunos colores
 
NEGRO = (0, 0, 0)
BLANCO = (255, 255, 255)
ROJO = (255, 0, 0)


            
            
class Bloque(pygame.sprite.Sprite):
    nombre=""
    puntos=0
    """
    Esta clase representa la pelota.        
    Deriva de la clase "Sprite" en Pygame
    """
    def __init__(self, color, width, height):
        """ Constructor. Le pasa el color al bloque, 
        así como la posición de x,y """
        # Llama a la clase constructor padre (Sprite)
        super().__init__() 
  
        # Crea una imagen del bloque y lo rellena de color.
        # También podríamos usar una imagen guardada en disco.
        self.image = pygame.Surface([width, height])
        self.image.fill(color)
  
        # Extraemos el objeto rectángulo que posee las dimensiones de la imagen.
        # Estableciendo los valores para rect.x and rect.y actualizamos la posición de este objeto.
        self.rect = self.image.get_rect()
         
    def reset_pos(self):
        """ Restablecemos la posición en la parte alta de la pantalla, en una ubicación aleatoria x.
        Si hubiera una colisión, sería llamada por update() o por el bucle principal.
        """
        self.rect.y = random.randrange(-300, -20)
        self.rect.x = random.randrange(700-20)        
 
    def update(self):
        """ Llamada para cada fotograma. """
 
        # Desplaza el bloque un píxel hacia abajo.
        self.rect.y += 1
         
        # Si el bloque estuviera muy abajo, lo restablecemos a la parte superior de la pantalla.
        if self.rect.y > 410:
            self.reset_pos()

class Protagonista(Bloque):
    """ La clase protagonista deriva de Bloque, pero sobrescribe su funcionalidad de 'update' 
    por una nueva función de desplazamiento que moverá el bloque con el ratón. """
    def update(self):
        # Obtiene la posición actual del ratón. La devuelve en forma de 
        # una lista de dos números.
        pos = pygame.mouse.get_pos()
          
        # Extraemos x e y de la lista, tal como si extrajéramos letras de una cadena de texto (string).
        # Coloca al objeto protagonista en la posición del ratón.
        self.rect.x = pos[0]
        self.rect.y = pos[1]        
         
# Iniciamos Pygame
pygame.init()

neo4j = Neo4jService('bolt://localhost:7687', 'neo4j', 'cuenca')
with neo4j._driver.session() as session:

    session.write_transaction(neo4j.crear_juego , "Bloques")
    
# Establecemos el largo y alto de la pantalla
largo_pantalla = 800
alto_pantalla = 400

pantalla = pygame.display.set_mode([largo_pantalla, alto_pantalla])
  
# Esta es una lista de 'sprites.' Cada bloque en el programa es 
# añadido a esta lista. La lista bes gestionada por la clase llamada  'Group.'
bloque_lista = pygame.sprite.Group()
  
# Esta es una lista de cada sprite, así como de los bloques y del protagonista.rojo
listade_todoslos_sprites= pygame.sprite.Group()
  
for i in range(50):
    # Esto representa un bloque
    bloque = Bloque(NEGRO, 10, 8)
  
    # Establece una ubicación aleatoria para el bloque
    bloque.rect.x = random.randrange(largo_pantalla)
    bloque.rect.y = random.randrange(alto_pantalla)
      
    # Añade el bloque a la lista de objetos
    bloque_lista.add(bloque)
    listade_todoslos_sprites.add(bloque)
      
        
#  Crea un bloque protagonista de color rojo
protagonista = Protagonista(ROJO, 20, 25)
listade_todoslos_sprites.add(protagonista)
  
# Iteramos hasta que el usuario haga click sobre el botón de salir.
hecho = False
  
# Lo usamos para gestionar cuan rápido se actualiza la pantalla
reloj = pygame.time.Clock()
  
puntuacion = 0
velocidad = 50
aux=20
nivel=1
nombreJ="Sibri"
# -------- Bucle Principal del Programa -----------
while not hecho:
    for evento in pygame.event.get(): 
        if evento.type == pygame.QUIT:
            hecho = True
    #Limpiamos la pantalla
    pantalla.fill(BLANCO)
     
    # Llamamos al método update() para cada sprite en la lista
    listade_todoslos_sprites.update()
      
    # Observamos si el bloque protagonista ha colisionado con algo
    lista_impactos_bloques = pygame.sprite.spritecollide(protagonista, bloque_lista, False)  
      
    # Comprobamos la lista de colisiones
    for bloque in lista_impactos_bloques:
        puntuacion += 1
        print("puntos" +str(puntuacion ))
        
        
        if puntuacion == aux:
            nivel=nivel+1
            velocidad=velocidad+50
            aux=aux+20
            print("nivel"+str(nivel))
            
         
        # Restablecemos al bloque, en la parte alta de la pantalla, para que vuelva caer.
        bloque.reset_pos() 

          
    # Dibujamos todos los sprites
    listade_todoslos_sprites.draw(pantalla)
      
    # Limitamos a 20 fotogramas por segundo 60
    reloj.tick(velocidad)
    
   
  
    # Avanzamos y actualizamos la pantalla con todo lo que hemos dibujado.
    pygame.display.flip()

    if puntuacion==20:
        neo4j = Neo4jService('bolt://localhost:7687', 'neo4j', 'cuenca')
        with neo4j._driver.session() as session:

            session.write_transaction(neo4j.crear_jugador,nombre,puntuacion,nivelJ)
            session.write_transaction(neo4j.crear_relacion_JuegoUS, "Bloques",nombreJ)
        pygame.quit()
pygame.quit()

puntos1
puntos2
puntos3
puntos4
puntos5
puntos6
puntos7
puntos8
puntos9
puntos10
puntos11
puntos12
puntos13
puntos14
puntos15
puntos16
puntos17
puntos18
puntos19
puntos20
nivel2


ServiceUnavailable: Failed to establish connection to IPv4Address(('127.0.0.1', 7687)) (reason [WinError 10061] No se puede establecer una conexión ya que el equipo de destino denegó expresamente dicha conexión)

In [ ]:
"""
Sample Python/Pygame Programs
Simpson College Computer Science
http://programarcadegames.com/
http://simpson.edu/computer-science/
"""
 
import pygame
import random
  
# Definimos algunos colores
NEGRO = (0, 0, 0)
BLANCO = (255, 255, 255)
ROJO = (255, 0, 0)
  

class Bloque(pygame.sprite.Sprite):  
    """
    Esta clase representa la pelota.        
    Deriva de la clase "Sprite" en Pygame
    """
     
    def __init__(self, color, width, height):
        """Constructor. Le pasa el color al bloque,
        así como la posición de x,y """
        # Llama a la clase constructor padre (Sprite)
        super().__init__()
 
        # Crea una imagen del bloque y lo rellena de color.
        # También podríamos usar una imagen guardada en disco.
        self.image = pygame.Surface([width, height])
        self.image.fill(color)
  
        # Extraemos el objeto rectángulo que posee las dimensiones
        # de la imagen.
        # Estableciendo los valores para rect.x and rect.y actualizamos
        # la posición de este objeto.
        self.rect = self.image.get_rect()
 
        # Variables de instancia que controlan los bordes
        # donde rebotamos
        self.limite_izquierdo = 0
        self.limite_derecho = 0
        self.limite_superior = 0
        self.limite_inferior = 0
 
        # Variables de instancia que controlan nuestras
        # velocidades y dirección actuales
        self.cambio_x = 0
        self.cambio_y = 0
            
 
    # Llamada para cada fotograma.
    def update(self):
        self.rect.x += self.cambio_x
        self.rect.y += self.cambio_y
         
        if self.rect.right >= self.limite_derecho or self.rect.left <= self.limite_izquierdo:
            self.cambio_x *= -1
 
        if self.rect.bottom >= self.limite_inferior or self.rect.top <= self.limite_superior:
            self.cambio_y *= -1
             
class Protagonista(Bloque):
    """ La clase protagonista deriva de Bloque, pero sobrescribe su
    funcionalidad de 'update' por una nueva función de desplazamiento
    que moverá el bloque con el ratón. """
    def update(self):
        # Obtiene la posición actual del ratón. La devuelve en forma de 
        # una lista de dos números.
        pos = pygame.mouse.get_pos()
          
      # Extraemos x e y de la lista, tal como si extrajéramos letras de una cadena de texto (string).
      # Coloca al objeto protagonista en la posición del ratón.
        self.rect.x = pos[0]
        self.rect.y = pos[1]        
         
#Iniciamos Pygame
pygame.init()
  
# Establecemos el largo y alto de la pantalla
largo_pantalla = 700
alto_pantalla = 400
pantalla=pygame.display.set_mode([largo_pantalla,alto_pantalla])
  
# Esta es una lista de 'sprites.' Cada bloque en el programa es 
# añadido a esta lista. La lista bes gestionada por la clase llamada  'Group.'
bloque_lista = pygame.sprite.Group()
  
# This is a list of every sprite. All bloques and the protagonista bloque as well.
listade_todoslos_sprites = pygame.sprite.Group()
  
for i in range(50):
    #  Esto representa un bloque
    bloque = Bloque(NEGRO, 20, 15)
  
    # Establece una ubicación aleatoria para el bloque
    bloque.rect.x = random.randrange(largo_pantalla)
    bloque.rect.y = random.randrange(alto_pantalla)
     
    bloque.cambio_x = random.randrange(-3,4)
    bloque.cambio_y = random.randrange(-3,4)
    bloque.limite_izquierdo = 0
    bloque.limite_superior = 0
    bloque.limite_derecho = largo_pantalla
    bloque.limite_inferior = alto_pantalla
     
    #Añade el bloque a la lista de objetos
    bloque_lista.add(bloque)
    listade_todoslos_sprites.add(bloque)
      

# Crea un bloque protagonista de color rojo
protagonista = Protagonista(ROJO, 20, 15)
listade_todoslos_sprites.add(protagonista)
  
#Iteramos hasta que el usuario haga click sobre el botón de salir.
hecho = False
  
# Lo usamos para gestionar cuan rápido se actualiza la pantalla
reloj = pygame.time.Clock()
  
puntuacion = 0
  
# --------  Bucle Principal del Programa  -----------
while not hecho:
    for evento in pygame.event.get(): 
        if evento.type == pygame.QUIT: 
            hecho = True
  
    # Limpiamos la pantalla
    pantalla.fill(BLANCO)
     
    # Llamamos al método update() para cada sprite en la lista
    listade_todoslos_sprites.update()
      
    # Observamos si el bloque protagonista ha colisionado con algo
    bloques_hit_list = pygame.sprite.spritecollide(protagonista, bloque_lista, True)  
      
    #Comprobamos la lista de colisiones
    for bloque in bloques_hit_list:
        puntuacion += 1
        print( puntuacion )
         
    # Dibujamos todos los sprites
    listade_todoslos_sprites.draw(pantalla)
      
    # Limitamos a 60 fotogramas por segundo
    reloj.tick(60)
  
    #Avanzamos y actualizamos la pantalla con todo lo que hemos dibujado.
    pygame.display.flip()

pygame.quit()

In [ ]:
from easyAI import TwoPlayersGame, Human_Player, AI_Player, Negamax
from flask import Flask, render_template_string, request, make_response


class TicTacToe(TwoPlayersGame):
    """ The board positions are numbered as follows:
            7 8 9
            4 5 6
            1 2 3
    """

    def __init__(self, players):
        self.players = players
        self.board = [0 for i in range(9)]
        self.nplayer = 1  # player 1 starts.

    def possible_moves(self):
        return [i + 1 for i, e in enumerate(self.board) if e == 0]

    def make_move(self, move):
        self.board[int(move) - 1] = self.nplayer

    def unmake_move(self, move):  # optional method (speeds up the AI)
        self.board[int(move) - 1] = 0

    WIN_LINES = [
        [1, 2, 3], [4, 5, 6], [7, 8, 9],  # horiz.
        [1, 4, 7], [2, 5, 8], [3, 6, 9],  # vertical
        [1, 5, 9], [3, 5, 7]  # diagonal
    ]

    def lose(self, who=None):
        """ Has the opponent "three in line ?" """
        if who is None:
            who = self.nopponent
        wins = [all(
            [(self.board[c - 1] == who) for c in line]
        ) for line in self.WIN_LINES]
        return any(wins)

    def is_over(self):
        return (self.possible_moves() == []) or self.lose() or \
            self.lose(who=self.nplayer)

    def show(self):
        print ('\n' + '\n'.join([
            ' '.join(
                [['.', 'O', 'X'][self.board[3 * j + i]] for i in range(3)]
            )
            for j in range(3)
        ]))

    def spot_string(self, i, j):
        return ["_", "O", "X"][self.board[3 * j + i]]

    def scoring(self):
        opp_won = self.lose()
        i_won = self.lose(who=self.nplayer)
        if opp_won and not i_won:
            print("0")
            return -100
        if i_won and not opp_won:
            print("100")
            return 100
        
        return 0

    def winner(self):
        if self.lose(who=2):
            return "AI Wins"
        return "Tie"


TEXT = '''
<!doctype html>
<html>
  <head><title>Tic Tac Toe</title></head>
  <body>
    <h1>Tic Tac Toe</h1>
    <h2>{{msg}}</h2>
    <form action="" method="POST">
      <table>
        {% for j in range(2, -1, -1) %}
        <tr>
          {% for i in range(0, 3) %}
          <td>
            <button type="submit" name="choice" value="{{j*3+i+1}}"
             {{"disabled" if ttt.spot_string(i, j)!="_"}}>
              {{ttt.spot_string(i, j)}}
            </button>
          </td>
          {% endfor %}
        </tr>
        {% endfor %}
      </table>
      <button type="submit" name="reset">Start Over</button>
    </form>
  </body>
</html>
'''

app = Flask(__name__)
ai_algo = Negamax(6)


@app.route("/", methods=['GET', 'POST'])
def play_game():
    ttt = TicTacToe([Human_Player(), AI_Player(ai_algo)])
    game_cookie = request.cookies.get('game_board')
    if game_cookie:
        ttt.board = [int(x) for x in game_cookie.split(",")]
    if "choice" in request.form:
        ttt.play_move(request.form["choice"])
        if not ttt.is_over():
            ai_move = ttt.get_move()
            ttt.play_move(ai_move)
    if "reset" in request.form:
        ttt.board = [0 for i in range(9)]
    if ttt.is_over():
        msg = ttt.winner()
    else:
        msg = "play move"
    resp = make_response(render_template_string(TEXT, ttt=ttt, msg=msg))
    c = ",".join(map(str, ttt.board))
    resp.set_cookie("game_board", c)
    return resp


if __name__ == "__main__":
    app.run()